In [13]:
import pandas as pd
from scipy.sparse import hstack
import spacy
import re
nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words -= {"not", "do"}

In [33]:
# Forming a dataframe
import sqlite3

# Connect to the database file
conn = sqlite3.connect("IMDB_Movies_2021.db")

# Create a cursor object
cursor = conn.cursor()

# Replace 'your_table_name' with the actual table name
table_name = "REVIEWS"

# Read the table into a DataFrame
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# Close the connection
conn.close()

In [35]:
df.info()
df.shape
df.columns
df.describe()
df.isnull().sum()
df = df.dropna()
df["RATING"].value_counts()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5450 entries, 0 to 5449
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      5450 non-null   int64  
 1   REVIEW  5450 non-null   object 
 2   RATING  5332 non-null   float64
 3   AUTHOR  5450 non-null   object 
 4   TITLE   5450 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 213.0+ KB


ID        0
REVIEW    0
RATING    0
AUTHOR    0
TITLE     0
dtype: int64

In [5]:
df["Class"] = df["RATING"].apply(lambda x: 0 if x < 4 else (1 if x in [4, 5, 6] else 2))
df.drop("AUTHOR", axis=1, inplace=True)
df.head()


,ID,REVIEW,RATING,TITLE,Class
0,1,I don't get all the terrible reviews for this ...,5.0,Not Bad\n,1
1,2,I cannot believe anyone could give this film l...,8.0,What are all the bad reviews about is it a wo...,2
2,3,Great White is not the worst way to spend 90 m...,4.0,Great White=Jaws Lite\n,1
3,4,Great White is as basic of a killer shark film...,4.0,Bare-bones killer shark film\n,1
4,5,"Terrible story, dialogue and CGI. The film has...",4.0,"Terrible story, dialogue, and CGI\n",1


(21238,)

In [159]:
from spellchecker import SpellChecker

In [37]:

#Ignore this function for now!!!!!

def fix_spelling(text):
    spell = SpellChecker()
    
    # Tokenize the text
    words = text.split()
    
    # Correct spelling for each word, they are in a list
    corrected_words = []
    for word in words:
        # Skip non-alphabetic words like numbers or punctuation, or if the word is None
        if word.isalpha():
            corrected_word = spell.correction(word)  # Get the corrected word
            if corrected_word is None:
                corrected_words.append(word)  # If correction is None, append the original word
            else:
                corrected_words.append(corrected_word)  # Otherwise, append the corrected word
        else:
            corrected_words.append(word)
    
    # Rebuild the text with corrected words
    corrected_text = " ".join(corrected_words)
    
    return corrected_text

In [200]:
def preprocess(text):
    doc = nlp(text)
    
    no_stop_words = [
        token.lemma_
        for token in doc
        if (
            not token.is_stop  # Remove stop words
            and not token.is_punct  # Remove punctuation
            and token.is_alpha  # Remove numbers, symbols
            and len(token.lemma_) > 2  # Remove short words
            and token.pos_ != "PROPN"  # Remove proper nouns
            and not re.match(r"https?://|www\.", token.text)  # Remove URLs
        )
    ]
    
    return " ".join(no_stop_words)
        
        
    

In [206]:
df["FIXED_REVIEW"] = df["REVIEW"].apply(preprocess)

A


In [203]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[["FIXED_REVIEW", "RATING"]], df.Class)

In [209]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

bow = v.fit_transform(X_train.FIXED_REVIEW.values)


<4087x11449 sparse matrix of type '<class 'numpy.int64'>'
	with 141752 stored elements in Compressed Sparse Row format>

In [232]:
X_rating = X_train["RATING"].values.reshape(-1, 1)  
X_combined = hstack([bow, X_rating])

array([[ 0.,  0.,  0., ...,  0.,  0.,  2.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.],
       [ 0.,  0.,  0., ...,  0.,  0.,  2.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0., 10.],
       [ 0.,  0.,  0., ...,  0.,  0., 10.]])

<4087x11449 sparse matrix of type '<class 'numpy.int64'>'
	with 141752 stored elements in Compressed Sparse Row format>